In [ ]:
# Importo pandas y matplotlib
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Leo el csv que contiene el dataset
df = pd.read_csv('../attacks.csv')
df.head()
# Saco el numero de columnas y filas del dataframe original
df.shape

In [ ]:
# Saco el numero total de valores nulos
df.isnull().sum().sum()

In [ ]:
# Saco el numero de valores nulos por columnas
null_cols = df.isnull().sum()
null_cols[null_cols > 0]

In [ ]:
# Saco el numero de valores no nulos por columna
not_null_cols = df.notnull()
not_null_cols.sum()

In [ ]:
# Renombro columnas
df = df.rename(columns={"Unnamed: 22":"Unnamed1"})
df = df.rename(columns={"Unnamed: 23":"Unnamed2"})

In [ ]:
# Elimino Unnamed1 y Unnamed 2 por irrelevantes basado en el hecho de que solo tenian 1 o 2 valores no nulos y tambien por irrelevantes para mi hipotesis
df = df.drop("Unnamed1", axis=1)
df = df.drop("Unnamed2", axis=1)

In [ ]:
# Elimino mas columnas por las mismas razones
df = df.drop(["href", "href formula", "pdf", "Date", "Sex ", "Investigator or Source", "Case Number", "Case Number.1", "Case Number.2", "Time", "Location", "original order", "Injury", "Name", "Age", "Area", "Year", "Country", ], axis=1)

In [ ]:
# Vuelvo a ver cuantos valores no nulos hay en las columnas con las que me he quedado
not_null_cols = df.notnull()
not_null_cols.sum()

In [ ]:
# Renombro esta columna para que sea mas facil referenciarla
df = df.rename(columns={"Fatal (Y/N)":"Fatal"})

In [ ]:
# Con esta tambien hago lo mismo
df = df.rename(columns={"Species ":"Species"})

In [ ]:
# Cambio el tipo de datos de la columna Fatal para evitar futuros errores potenciales conociendo el tipo de datos que contienen
df["Fatal"] = df["Fatal"].astype(str)
fatal_col_nulls = df["Fatal"].isnull().sum()
fatal_col_not_nulls = df["Fatal"].notnull().sum()
# Veo cuantos valores nulos y no nulos tienen para hacerme una idea
print("Fatal total null values:", fatal_col_nulls)
print("Fatal total not null values:", fatal_col_not_nulls)

In [ ]:
# Hago un set para agrupar solo cada valor distinto y al saber que esta columna solo deberia admitir dos valores (Y, N)
# Me doy cuenta que hay mas valores distintos de los que me sirven
fatal_set = set(df["Fatal"])
print(fatal_set)

In [ ]:
# Por otra parte uso el mismo approach para investigar la columnas Species
df["Species"].value_counts()
species_set = set(df["Species"])
species_set
df["Species"] = df["Species"].astype(str)

In [ ]:
# Me doy cuenta que en la columna Species hay ciertas filas con valor "Shark involment not confirmed"
# Procedo a eliminar dichas filas que contengan ese valor para mitigar la ambiguedad en mi dataset
# Tambien procedo a eliminar filas vacias en esa columna y registros de ataques que no sean de tiburon
df = df[df.Species != "Shark involvement not confirmed"]
df = df[df.Species != "nan"]
df = df[df.Species != "Lesser spotted dogfish, Scyliorhinus canicula, less than 80 cm in length"]
df = df[df.Species != "NaN"]
df["Species"].value_counts()

In [ ]:
# Hago lo mismo con la columna Fatal y veo que hay mas de dos posibles valores distintos 
# y cuantos registros hay de cada valor distinto
df["Fatal"].value_counts()

In [ ]:
# Procedo a cambiar los registros mal escritos pero cuyo valor no es ambiguo
df["Fatal"] = df["Fatal"].str.replace(" N","N")
df["Fatal"] = df["Fatal"].str.replace("N ","N")
df["Fatal"] = df["Fatal"].str.replace("n","N")

In [ ]:
# Procedo a eliminar las filas cuyos valores aportan ambiguedad
# por ejemplo "F" podria ser "Fatal" o "False"
df = df[df.Fatal != "UNKNOWN"]
df = df[df.Fatal != "nan"]
df = df[df.Fatal != "F"]
df = df[df.Fatal != "#VALUE!"]
df = df[df.Fatal != "NaN"]

In [ ]:
# ahora imprimo los valores distintos y su ocurrencia y veo que me he quedado con dos valores distintos
# que era lo que me interesaba
df["Fatal"].value_counts()

In [ ]:
# Uso el metodo describe para ver un poco la descripcion estadistica y asegurarme de que no se me haya colado otro valor
# Notese que pone "unique 2" por lo cual todo bajo control
df["Fatal"].describe()

In [ ]:
# Veo como va quedando el dataframe
df

In [ ]:
# Procedo a hacer lo mismo con la columna Type
df["Type"].value_counts()

In [ ]:
# Procedo a eliminar los valores ambiguos
df = df[df.Type != "Sea Disaster"]
df = df[df.Type != "Invalid"]
df = df[df.Type != "Boat"]
df = df[df.Type != "Boating"]
df["Type"].value_counts()

In [ ]:
# Veo un poco la estadistica descriptiva de la columna Species y me doy cuenta de que hay muchos valores distintos
df["Species"].describe()

In [ ]:
# Imprimo las ocurrencias de cada valor unico y veo que en cierto punto comienza a haber un desastre
df["Species"].value_counts()

In [ ]:
# Comienzo a reemplazar strings que contengan informacion concreta sobre la especie de tiburon involucrado en el ataque
df["Species"] = df["Species"].str.replace("4.2 m white shark", "White shark")

In [ ]:
# Hago e imprimo un set para ver con detalle cuales son esos miles de valores unicos en esa columna
set(df["Species"])

In [ ]:
# Aca comienzo a limpiar manualmente cada valor unico y reasignarlo a su especie
# Uso el metodo str.startswith() para ahorrarme cierta cantidad de trabajo donde el nombre del tiburon este al principio
df.loc[df['Species'].str.startswith('Bull'), "Species"] = 'Bull shark'
df.loc[df['Species'].str.startswith('Tiger'), "Species"] = 'Tiger shark'
df.loc[df['Species'].str.startswith('White shark'), "Species"] = 'White shark'
df.loc[df['Species'].str.startswith('Lemon'), "Species"] = 'Lemon shark'
df.loc[df['Species'].str.startswith('Zambesi'), "Species"] = 'Zambesi shark'
df.loc[df['Species'].str.startswith('Blue'), "Species"] = 'Blue shark'
df.loc[df['Species'].str.startswith('\"sand\"'), "Species"] = 'Sand shark'
df.loc[df['Species'].str.startswith(' Bull'), "Species"] = 'Bull shark'
df.loc[df['Species'].str.startswith('Bronze'), "Species"] = 'Bronze whaler shark'
df.loc[df['Species'].str.startswith('Hammerhead'), "Species"] = 'Hammerhead shark'
df.loc[df['Species'].str.startswith('Raggedtooth'), "Species"] = 'Raggedtooth shark'
df.loc[df['Species'].str.startswith(' Sandbar'), "Species"] = 'Sandbar shark'
df.loc[df['Species'].str.startswith(' Sevengill'), "Species"] = 'Sevengill shark'
df.loc[df['Species'].str.startswith(' Raggedtooth'), "Species"] = 'Raggedtooth shark'
df.loc[df['Species'].str.startswith(' Tiger'), "Species"] = 'Tiger shark'
df.loc[df['Species'].str.startswith(' White'), "Species"] = 'White shark'
df.loc[df['Species'].str.startswith(' Blacktip'), "Species"] = 'Blacktip shark'
df.loc[df['Species'].str.startswith(' Galapagos'), "Species"] = 'Galapagos shark'
df.loc[df['Species'].str.startswith(' White'), "Species"] = 'White shark'
df.loc[df['Species'].str.startswith('Carpet'), "Species"] = 'Carpet shark'
df.loc[df['Species'].str.startswith('Caribbean'), "Species"] = 'Caribbean reef shark'
df.loc[df['Species'].str.startswith('Copper'), "Species"] = 'Copper shark'
df.loc[df['Species'].str.startswith('Dusky'), "Species"] = 'Dusky shark'
df.loc[df['Species'].str.startswith('Grey reef'), "Species"] = 'Grey reef shark'
df.loc[df['Species'].str.startswith('Grey nurse'), "Species"] = 'Grey nurse shark'
df.loc[df['Species'].str.startswith('Oceanic'), "Species"] = 'Oceanic whitetip shark'
df.loc[df['Species'].str.startswith('Wobbegong'), "Species"] = 'Wobbegong whitetip shark'
df.loc[df['Species'].str.startswith('Zambesi'), "Species"] = 'Zambesi shark'
df.loc[df['Species'].str.startswith('Unknown'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Whitetip'), "Species"] = 'Whitetip reef shark'
df.loc[df['Species'].str.startswith('Whaler'), "Species"] = 'Whaler shark'
df.loc[df['Species'].str.startswith('reef'), "Species"] = 'Reef shark'
df.loc[df['Species'].str.startswith('Thought'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Spinner'), "Species"] = 'Spinner shark'
df.loc[df['Species'].str.startswith('Tawny'), "Species"] = 'Tawny shark'
df.loc[df['Species'].str.startswith(' Tawney'), "Species"] = 'Tawny shark'
df.loc[df['Species'].str.startswith('Mako'), "Species"] = 'Mako shark'
df.loc[df['Species'].str.startswith('Nurse'), "Species"] = 'Nurse shark'
df.loc[df['Species'].str.startswith('Reef'), "Species"] = 'Reef shark'
df.loc[df['Species'].str.startswith('Sand'), "Species"] = 'Sand shark'
df.loc[df['Species'].str.startswith('Unidentified'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Possibly'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Said'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Sevengill'), "Species"] = 'Sevengill shark'
df.loc[df['Species'].str.startswith('Silky'), "Species"] = 'Silky shark'
df.loc[df['Species'].str.startswith('Species'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Spurdog'), "Species"] = 'Spurdog shark'
df.loc[df['Species'].str.startswith('Seven-gill'), "Species"] = 'Sevengill shark'
df.loc[df['Species'].str.startswith('Reported'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Remains'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Shortfin'), "Species"] = 'Mako shark'
df.loc[df['Species'].str.startswith('Zambezi'), "Species"] = 'Zambesi shark'
df.loc[df['Species'].str.startswith('said'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('possibly'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('Though'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('dogfish'), "Species"] = 'Dogfish shark'
df.loc[df['Species'].str.startswith('bull'), "Species"] = 'Bull shark'
df.loc[df['Species'].str.startswith('Bull'), "Species"] = 'Bull shark'
df.loc[df['Species'].str.startswith('nurse'), "Species"] = 'Nurse shark'
df.loc[df['Species'].str.startswith('sandtiger'), "Species"] = 'Sandtiger shark'
df.loc[df['Species'].str.startswith('sand'), "Species"] = 'Sand shark'
df.loc[df['Species'].str.startswith('Undefined'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('reef'), "Species"] = 'Reef shark'
df.loc[df['Species'].str.startswith(' reef'), "Species"] = 'Reef shark'
df.loc[df['Species'].str.startswith(' white'), "Species"] = 'White shark'
df.loc[df['Species'].str.startswith(' a small shark'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('\"A 2\' (0.6 m) brown shark\"'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('\"A long thin brown-colored shark\"'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('\"A pack of sharks\"'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith('\"A small shark\"'), "Species"] = 'Unidentified species'
df.loc[df['Species'].str.startswith(' Unidentified'), "Species"] = 'Unidentified species'

In [ ]:
set(df['Species'])

In [ ]:
# Aqui uso el metodo str.replace() para reemplazar valores mas complejos de extraer
df["Species"] = df["Species"].str.replace("1 m hammerhead shark", "Hammerhead shark")
df["Species"] = df["Species"].str.replace("1.5 m to 1.8 m [5' to 6']  blacktip shark", "Blacktip shark")
df["Species"] = df["Species"].str.replace("1.5 m to 1.8 m [5' to 6'] Zambesi shark", "Zambesi shark")
df["Species"] = df["Species"].str.replace("1.5 m to 1.8 m [5' to 6'] sandbar shark", "Sandbar shark")
df["Species"] = df["Species"].str.replace("\"a black-tipped shark\"", "Blacktip shark")
df["Species"] = df["Species"].str.replace("\"a blue shark\"", "Blue shark")
df["Species"] = df["Species"].str.replace("\"a dog shark\"", "Dog shark")
df["Species"] = df["Species"].str.replace("\"a little shark\"", "Unidentified species")
df["Species"] = df["Species"].str.replace("\"a large shark\"", "Unidentified species")
df["Species"] = df["Species"].str.replace("\"a small shark\"", "Unidentified species")
df["Species"] = df["Species"].str.replace("\"a young shark\"", "Unidentified species")
df["Species"] = df["Species"].str.replace("\"reef shark\"", "Reef shark")
df["Species"] = df["Species"].str.replace(" \"gummy\" shark (Rhizoprionodon or Loxodon) 1.2 m [4\']", "Unidentified species")
df["Species"] = df["Species"].str.replace("\"black tipped\" shark", "Black-tipped shark")
df["Species"] = df['Species'].str.replace('\"gray shark\"', 'Grey shark')
df["Species"] = df['Species'].str.replace('\"grey shark\"', 'Grey shark')
df["Species"] = df['Species'].str.replace('  "gummy" shark (Rhizoprionodon or Loxodon) 1.2 m [4\']', 'Unidentified species')
df["Species"] = df['Species'].str.replace('Juvenile dusky or blacktip shark', 'Unidentified species')
df["Species"] = df['Species'].str.replace('Wobbegong shark?', 'Unidentified species')
df["Species"] = df['Species'].str.replace('Zambezi shark (tooth fragments recovered)',  'Zambesi shark')
df["Species"] = df['Species'].str.replace('Whtietip reef shark',  'Whitetip reef shark')
df["Species"] = df['Species'].str.replace('Undefined shark',  'Unidentified species')
df["Species"] = df['Species'].str.replace('Unidentified shark',  'Unidentified species')
df["Species"] = df['Species'].str.replace('Undefined species',  'Unidentified species')
df["Species"] = df['Species'].str.replace('a sand shark',  'Sand shark')
df["Species"] = df['Species'].str.replace('a small reef shark',  'Reef shark')
df["Species"] = df['Species'].str.replace('\"Blue whaler\" (Galeolamna)',  'Unidentified species')
df["Species"] = df['Species'].str.replace('\"sand shark\"',  'Sand shark')

In [ ]:
# Veo que el set se redujo mucho pero aun no es lo ideal y me rindo
set(df["Species"])


In [ ]:
# Aqui veo cuales son las ocurrencias por cada especie unica y me veo que la suma de los 5 primeros
# es mas de 600 lo cual me aperece adecuado
# ya que es impensable que los 2500 ataques hayan sido provocados por 2500 distintas especies de tiburon
df["Species"].value_counts()

In [ ]:
# Aqui me propongo a sacar un piechart de las primeras 5 especies con mayor ocurrencias de ataques
# Lo cual constituye mi primera hipotesis
labels = 'White shark', 'Tiger shark', 'Unidentified species', 'Bull shark', "Bronze whaler shark"
sizes = [333, 198, 152, 119, 55]
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', "purple"]
explode = (0, 0, 0, 0, 0) 

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=False, startangle=140)

plt.axis('equal')
plt.show()
# Como se aprecia en el grafico generado el tiburon blanco es el que tiene mas ocurrencias de ataques.

In [ ]:
# Aqui veo que valores unicos tienen las filas de la columna Activity
# y la elimino por irrelevante con respecto a mis objetivos
Activity_set = set(df["Activity"])
Activity_set
df = df.drop("Activity", axis=1)

In [ ]:
# Imprimo la descripcion estadistica de la columna Type
df["Type"].describe()

In [ ]:
# Hago lo mismo con la columna Fatal
df["Fatal"].describe()

In [ ]:
# Elimino la columna species despues de haber logrado el grafico final
df = df.drop("Species", axis=1)

In [ ]:
# Imprimo una descriptcion estadistica de todo el dataframe
df.describe()

In [ ]:
# Hago un dataframe subconjunto del original pero filtrado en base a ciertas condiciones
# En este caso particular quiero los ataques no provocado y no fatales
unprov_not_fatal = df[(df["Type"] == "Unprovoked") & (df["Fatal"] == "N")]

In [ ]:
# Aqui veo el resultado
unprov_not_fatal.describe()

In [ ]:
# En otro dataframe quiero los ataques no provocados pero fatales
unprov_fatal = df[(df["Type"] == "Unprovoked") & (df["Fatal"] == "Y")]
unprov_fatal.describe()

In [ ]:
# Aqui quiero los provocados no fatales
prov_not_fatal = df[(df["Type"] == "Provoked") & (df["Fatal"] == "N")]
prov_not_fatal.describe()

In [ ]:
# y respectivamente los provocados y fatales
prov_fatal = df[(df["Type"] == "Provoked") & (df["Fatal"] == "Y")]
prov_fatal.describe()

In [ ]:
# Aqui procedo a graficar en base a la informacion de cada dataframe derivado del original
labels = 'Unprovoked not fatal', 'Unprovoked fatal', 'Provoked not fatal', 'Provoked fatal'
sizes = [1764, 381, 388, 6]
colors = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue']
explode = (0, 0, 0, 0.1)  # explode 1st slice

plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=False, startangle=140)

plt.axis('equal')
plt.show()

In [ ]:
# Basado en la grafica generada el 69.5% de los ataques son no fatales y no provocados
# y solamente un 0.2% corresponden a ataques fatales y provocados
# lo cual me proporciona informacion suficiente para creer (y concluir) que los tiburones son de naturaleza "reactiva"
# y no tienden a atacar a los seres humanos y si lo hacen luego huyen (por ende la gran cantidad de ataques no fatales)
# una especulacion basada en esta data es que tal vez los tiburones ataquen al ser humano por curiosidad y al ver que
# no es su tipo de alimento preferido se alejan, y el victimario podria sobrevivir o morir como consecuencia de 
# una hemorragia.
# por otro lado me hace pensar creer que "provocar" a un tiburon tiene que ver mas con interactuar con sus intereses
# como por ejemplo acercarse mucho a sus crias o interactuar con un animal que sea su alimento.